# AOP GEE Time Series Using the Python API

In [1]:
import ee, geemap

In [2]:
ee.Authenticate()

Enter verification code: 4/1AbUR2VN4ZGfN-bLQ0rl0WGp7x3CXT_y4M7WmGvXcuMBWr5MAfbqXVyLfjro

Successfully saved authorization token.


In [3]:
ee.Initialize()

In [16]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

First we need to set the NEON site and the years of data we want to pull in. 

For this example, we will pull in the NEON Great Smokey Mountain Site (GRSM) which currently has three years of data added to Earth Engine: 2016, 2017, and 2021. To see which years of data are available at GRSM, run the code chunk below.

In [43]:
# List all available sites in the NEON SDR image collection:
sdrCol = ee.ImageCollection('projects/neon-prod-earthengine/assets/DP3-30006-001')

print('NEON Images in the SDR Collection',sdrCol.aggregate_array('system:index').getInfo())

flightYears = sdrCol.aggregate_array('FLIGHT_YEAR').getInfo()
sites = sdrCol.aggregate_array('NEON_SITE').getInfo()

# print('NEON SDR years and sites:', siteYears)
print('\nData available in GEE for site',site,':')
print([year_site for year_site in zip(flightYears,sites) if site in year_site])

NEON Images in the SDR Collection ['2016_GRSM_2_SDR', '2016_HARV_3_SDR', '2017_GRSM_3_SDR', '2017_SERC_3_SDR', '2019_HARV_6_SDR', '2019_HEAL_3_SDR', '2019_JORN_3_SDR', '2019_SOAP_4_SDR', '2020_CPER_7_SDR', '2021_ABBY_4_SDR', '2021_GRSM_5_SDR', '2021_HEAL_4_SDR', '2021_JORN_4_SDR', '2021_SJER_5_SDR', '2021_SOAP_5_SDR']

Data available in GEE for site GRSM :
[(2016, 'GRSM'), (2017, 'GRSM'), (2021, 'GRSM')]


In [11]:
years = ee.List([2016, 2017, 2021])
print(years.getInfo())

site = 'GRSM'

[2016, 2017, 2021]


If you'd like to read in every year, you can use `ee.List.sequence(start_year, end_year)`, but since AOP data is not collected at every site every year, and all of the data has not yet been added, we recommend you check the data availability and hten use `ee.List` to include the data that are available.

In [12]:
# function to read the AOP SDR image collection, filter on a provided site (4-letter code,
# see https://www.neonscience.org/field-sites/explore-field-sites for all NEON sites)
# selects the data bands and masks to include only the clear-weather data (<10% cloud cover)
def yearly_sdr_image(year):

    start_date = ee.Date.fromYMD(year, 1, 1) 
    end_date = start_date.advance(1, "year")
    
    aop_sdr = ee.ImageCollection('projects/neon-prod-earthengine/assets/DP3-30006-001') \
        .filterDate(start_date, end_date) \
        .filterMetadata('NEON_SITE', 'equals', site).mosaic()
    
    # Read in only the data bands, all of which start with "B", eg. "B001"
    sdr_data = aop_sdr.select('B.*')
    
    # Extract Weather Quality Indicator layer
    weather_quality_band = aop_sdr.select(['Weather_Quality_Indicator']);

    # Select only the clear weather data (<10% cloud cover)
    clear_weather = weather_quality_band.eq(1); # 1 = 0-10% cloud cover
    
    # Mask out all cloudy pixels from the SDR data cube
    sdr_clear_weather = sdr_data.updateMask(clear_weather);

    return sdr_clear_weather

images = years.map(yearly_sdr_image)

# Set the visualization parameters so contrast is maximized, and display RGB bands (true-color image)
visParams = {'min':0,'max':1200,'gamma':0.9,'bands':['B053','B035','B019']};

for index in range(0, len(years.getInfo())):
    image = ee.Image(images.get(index))
    layer_name = site + " " + str(years.getInfo()[index]) + " SDR" 
    print('Adding ' + layer_name)
    Map.addLayer(image, visParams, layer_name)
    
# Center the map on the site
grsmCenter = ee.Geometry.Point([-83.5, 35.7]);
Map.centerObject(grsmCenter, 11);

Adding GRSM 2016 SDR
Adding GRSM 2017 SDR
Adding GRSM 2021 SDR


In [13]:
# function to read the AOP SDR image collection, filter on a provided site (4-letter code,
# see https://www.neonscience.org/field-sites/explore-field-sites for all NEON sites)
# selects the data bands and masks to include only the clear-weather data (<10% cloud cover)
def yearly_weather_band(year):

    start_date = ee.Date.fromYMD(year, 1, 1) 
    end_date = start_date.advance(1, "year")
    
#     print('filtering image collection by date and site')
    aop_sdr = ee.ImageCollection('projects/neon-prod-earthengine/assets/DP3-30006-001') \
        .filterDate(start_date, end_date) \
        .filterMetadata('NEON_SITE', 'equals', site).mosaic()
    
    # Extract Weather Quality Indicator band
    weather_quality_band = aop_sdr.select(['Weather_Quality_Indicator']);

    return weather_quality_band

weather_bands = years.map(yearly_weather_band)

# Define a palette for the weather - to match NEON AOP's weather color conventions (green-yellow-red)
gyrPalette = ['00ff00', # green (<10% cloud cover)
              'ffff00', # yellow (10-50% cloud cover)
              'ff0000' # red (>50% cloud cover)
             ];

# parameters to display the weather bands (cloud conditions) with the green-yellow-red palette
weather_visParams = {'min': 1, 'max': 3, 'palette': gyrPalette, 'opacity': 0.3};

# loop through the layers and add them to the Map
for index in range(0, len(years.getInfo())):
    weather_band = ee.Image(weather_bands.get(index))
    layer_name = site + " " + str(years.getInfo()[index]) + " Cloud Cover" 
    print('Adding ' + layer_name)
    Map.addLayer(weather_band, weather_visParams, layer_name, 0) # 0 means the layers will not be selected "on" by default
    
# Center the map on the site
# grsmCenter = ee.Geometry.Point([-83.5, 35.7]);
# Map.centerObject(grsmCenter, 11);

Adding GRSM 2016 Cloud Cover
Adding GRSM 2017 Cloud Cover
Adding GRSM 2021 Cloud Cover


Now let's add the full image collection (all years) at GRSM as a Map Layer. This is nice for using the time-lapse feature in the map window!

In [17]:
# Read in Surface Directional Reflectance (SDR) Image Collection
aopSDR = ee.ImageCollection('projects/neon-prod-earthengine/assets/DP3-30006-001')

# Filter by site to read in GRSM SDR image collection from all available years
grsmSDR = aopSDR.filterMetadata('NEON_SITE', 'equals', site);

# sdr2021 = aopSDR.filterDate('2021-01-01', '2021-12-31').filterMetadata('NEON_SITE', 'equals', site).mosaic();

# Read in only the data bands, all of which start with "B", eg. "B001"
grsmSDR_data = grsmSDR.select('B.*')

# weather_quality_band = grsmSDR.select(['Weather_Quality_Indicator']);

# Select only the clear weather data (<10% cloud cover)
# clear_weather = weather_quality_band.eq(1); # 1 = 0-10% cloud cover
    
# Mask out all cloudy pixels from the SDR data cube
# grsmSDR_clear_weather = grsmSDR_data.updateMask(clear_weather);

# Set the visualization parameters so contrast is maximized, and display RGB bands (true-color image)
visParams = {'min':0,'max':1200,'gamma':0.9,'bands':['B053','B035','B019']};

# Add the GRSM image collection as a layer to the Map
Map.addLayer(grsmSDR_data, visParams, 'GRSM SDR');

grsmCenter = ee.Geometry.Point([-83.5, 35.7]);
# Center the map on HEAL
Map.centerObject(grsmCenter, 11);